In [29]:
# EDEK
!pip install -q transformers rich torchmetrics sklearn pytorch-lightning

In [30]:
import pandas as pd
import numpy as np
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3
import dask.bag as db
import json
import zipfile
from sklearn.preprocessing import MultiLabelBinarizer
import csv
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertModel, BertConfig
from sklearn.model_selection import train_test_split
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import RichProgressBar
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from torchmetrics import F1
from torchmetrics.functional import accuracy
from transformers import RobertaTokenizer, RobertaModel, AutoTokenizer, AutoModel
from transformers import AutoModel, AdamW, get_cosine_schedule_with_warmup
import math
from torchmetrics.functional.classification import auroc
import torch.nn.functional as F
from torch import nn


In [31]:
!mkdir dataset

In [32]:
raw_json_data = "arxiv-metadata-oai-snapshot-json.zip"
pickled_dataset = 'dataset_pickle.pkl'
pickled_dataset_onehot = 'dataset_multilabel.pkl'

In [33]:
def __iter__(self): return 0

ibm_api_key_id = 'SECRET'
bucket = 'SECRET'

client_c2b4ce19d76d4b7c87bdf6c8c84d662c = ibm_boto3.client(service_name='s3',
    ibm_api_key_id=ibm_api_key_id,
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

streaming_body_1 = client_c2b4ce19d76d4b7c87bdf6c8c84d662c.get_object(Bucket=bucket, Key='arxiv-metadata-oai-snapshot.json.zip')['Body']

def save_to_cloud(key, file):
    client_c2b4ce19d76d4b7c87bdf6c8c84d662c.upload_file(Bucket=bucket, Key=f'snapshot-{file}', Filename=file)

    
def get_file(key, filename):
    client_c2b4ce19d76d4b7c87bdf6c8c84d662c.download_file(Bucket=bucket, Key=key, Filename=filename)

In [34]:
# try:
#     get_file('arxiv-metadata-oai-snapshot.pkl', 'dataset_pickle.pkl')
# except:
#     print("Exception while pulling, will recreate from scratch")
    

# if not os.path.exists(pickled_dataset):
#     get_file('arxiv-metadata-oai-snapshot.json.zip', raw_json_data)
#     with zipfile.ZipFile(raw_json_data, 'r') as zip_ref:
#         zip_ref.extractall("dataset")
#     docs = db.read_text('./dataset/arxiv-metadata-oai-snapshot.json').map(json.loads)
#     trim = lambda x: {'id': x['id'],
#                   'category':x['categories'].split(' '),
#                   'abstract':x['abstract'].replace('\t', ' ')}
#     docs_df = (docs.map(trim).compute())
#     df = pd.DataFrame(docs_df)
#     df.to_pickle(pickled_dataset)
#     save_to_cloud('arxiv-metadata-oai-snapshot.pkl', pickled_dataset)    

In [35]:
# df = pd.read_pickle('dataset_pickle.pkl')

In [36]:
# df

In [37]:
# try:
#     get_file('snapshot-dataset_multilabel.pkl', 'dataset_multilabel.pkl')
#     print("Got S3 file")
# except:
#     print("Exception while pulling, will recreate from scratch")
    
# if not os.path.exists('dataset_multilabel.pkl'):
#     mlb = MultiLabelBinarizer()
#     labels = mlb.fit_transform(df.category)
#     df_onehot = pd.concat([df[['abstract']], pd.DataFrame(labels)], axis=1)
#     df_onehot.columns = ['abstract'] + list(mlb.classes_)
#     df_onehot.to_pickle('dataset_multilabel.pkl')
#     save_to_cloud('snapshot-dataset_multilabel.pkl', 'dataset_multilabel.pkl')

In [38]:
# df_onehot = pd.read_pickle('dataset_multilabel.pkl')

In [39]:
# LABEL_COLUMNS = df_onehot.columns.tolist()[1:]
# LABEL_COLUMNS

## Trying preprocessing?

In [40]:
# get_file('arxiv-metadata-oai-snapshot.json.zip', raw_json_data)
# with zipfile.ZipFile(raw_json_data, 'r') as zip_ref:
#     zip_ref.extractall("dataset")
# docs = db.read_text('./dataset/arxiv-metadata-oai-snapshot.json').map(json.loads)


In [41]:
# get_latest_version = lambda x: x['versions'][-1]['created']
# trim = lambda x: {'id': x['id'],
#               'category':x['categories'].split(' '),
#               'abstract':x['abstract'].replace('\t', ' ')}
# docs_df = (docs.filter(lambda x: int(get_latest_version(x).split(' ')[3]) > 2009).map(trim).compute())
# df = pd.DataFrame(docs_df)
# df.to_pickle("df2009.pkl")
# save_to_cloud('df2009.pkl', 'df2009.pkl')

In [42]:
# get_file('snapshot-df2009.pkl', 'df2009.pkl')
# df = pd.read_pickle('df2009.pkl')

In [43]:
# %%time
# df["abstract"] = df["abstract"].apply(lambda text: " ".join([word for word in text.replace("\n", " ").replace("\t", " ").strip().split(" ") if len(word) > 0]))

In [44]:
# frequent_categories = pd.Series(np.concatenate(df['category'])).value_counts().reset_index(name="count").query("count > 1000")["index"].tolist()

In [45]:
# len(frequent_categories)

In [46]:
# df = df[df["category"].apply(lambda cats: all(elem in frequent_categories for elem in cats))]

In [47]:
# df = df[df["abstract"].apply(lambda text: len(text.split()) > 128)]

In [48]:
# df = df.reset_index()

In [49]:
# categories_filtered = pd.Series(np.concatenate(df['category'])).value_counts()

In [50]:
# from rich.progress import track

# # Select 5k randomly, then at least 1k from each category, then drop duplicates
# uni_df = df.sample(n = 5000)
# for category, _ in track(categories_filtered.items(), total=len(categories_filtered), description="Pulling 1k from each cat"):
#     subset_df = df[df["category"].apply(lambda cats: category in cats)]
#     uni_df = pd.concat([subset_df.sample(n = min(1000, len(subset_df))), uni_df], ignore_index=True, sort=False)

In [51]:
# uni_df = uni_df.iloc[uni_df.astype(str).drop_duplicates().index]


In [52]:
# len(uni_df)

In [53]:
# uni_df = uni_df.reset_index()

In [54]:
# mlb = MultiLabelBinarizer()
# labels = mlb.fit_transform(uni_df.category)
# df_onehot = pd.concat([uni_df[['abstract']], pd.DataFrame(labels)], axis=1)
# df_onehot.columns = ['abstract'] + list(mlb.classes_)


In [55]:
# df_onehot.to_pickle('uni_df.pkl')
# save_to_cloud('uni_df.pkl', 'uni_df.pkl')

## Model

In [56]:
get_file('snapshot-uni_df.pkl', 'uni_df.pkl')

In [57]:
df_onehot = pd.read_pickle('uni_df.pkl')
LABEL_COLUMNS = df_onehot.columns.tolist()[1:]

In [58]:
LABEL_COLUMNS

['astro-ph.CO',
 'astro-ph.EP',
 'astro-ph.GA',
 'astro-ph.HE',
 'astro-ph.IM',
 'astro-ph.SR',
 'cond-mat.dis-nn',
 'cond-mat.mes-hall',
 'cond-mat.mtrl-sci',
 'cond-mat.other',
 'cond-mat.quant-gas',
 'cond-mat.soft',
 'cond-mat.stat-mech',
 'cond-mat.str-el',
 'cond-mat.supr-con',
 'cs.AI',
 'cs.AR',
 'cs.CC',
 'cs.CE',
 'cs.CG',
 'cs.CL',
 'cs.CR',
 'cs.CV',
 'cs.CY',
 'cs.DB',
 'cs.DC',
 'cs.DL',
 'cs.DM',
 'cs.DS',
 'cs.ET',
 'cs.FL',
 'cs.GR',
 'cs.GT',
 'cs.HC',
 'cs.IR',
 'cs.IT',
 'cs.LG',
 'cs.LO',
 'cs.MA',
 'cs.MM',
 'cs.MS',
 'cs.NA',
 'cs.NE',
 'cs.NI',
 'cs.OH',
 'cs.PF',
 'cs.PL',
 'cs.RO',
 'cs.SC',
 'cs.SD',
 'cs.SE',
 'cs.SI',
 'cs.SY',
 'econ.EM',
 'econ.GN',
 'econ.TH',
 'eess.AS',
 'eess.IV',
 'eess.SP',
 'eess.SY',
 'gr-qc',
 'hep-ex',
 'hep-lat',
 'hep-ph',
 'hep-th',
 'math-ph',
 'math.AC',
 'math.AG',
 'math.AP',
 'math.AT',
 'math.CA',
 'math.CO',
 'math.CT',
 'math.CV',
 'math.DG',
 'math.DS',
 'math.FA',
 'math.GM',
 'math.GN',
 'math.GR',
 'math.GT',
 'ma

In [59]:
# Baseline?

In [60]:
# from sklearn.multiclass import OneVsRestClassifier
# from sklearn.pipeline import Pipeline
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.linear_model import LogisticRegression

# pipeline = Pipeline([
#     ('tfidf', TfidfVectorizer(analyzer='word', 
#                               smooth_idf=False,
#                               stop_words='english', 
#                               min_df = 3, 
#                               max_df = 0.95, 
#                               strip_accents='unicode', 
#                               sublinear_tf = True)),
#     ('clf', OneVsRestClassifier(LogisticRegression(multi_class='multinomial', max_iter=300, verbose=1, n_jobs=-1)))
# ])

In [61]:
# RANDOM_SEED = 1337
# df_train, df_test = train_test_split(df_onehot, test_size=0.1, random_state=RANDOM_SEED, shuffle=True)
# df_train, df_validation = train_test_split(df_train, test_size=0.2, random_state=RANDOM_SEED, shuffle=True)
# df_train = df_train.reset_index(drop=True)
# df_test = df_test.reset_index(drop=True)
# df_validation = df_validation.reset_index(drop=True)

In [62]:
# X_train = df_train["abstract"]
# y_train = df_train.loc[:, ~df_train.columns.isin(['abstract'])]
# X_test = df_test["abstract"]
# y_test = df_test.loc[:, ~df_test.columns.isin(['abstract'])]

In [63]:
# %%time
# pipeline.fit(X_train, y_train)

In [64]:
# preds_proba = pipeline.predict_proba(X_test)

In [65]:
# def predict(tresh = 0.5):
#     y_pred = [[int(val >= tresh) for val in pred] for pred in preds_proba]
#     y_true = y_test
#     print(classification_report(y_true, y_pred))
#     print("Acc score: ", accuracy_score(y_true, y_pred))

In [66]:
# predict()

In [67]:
# predict(0.2)

In [68]:
# predict(0.15)

In [69]:
# from sklearn.metrics import confusion_matrix
# y_pred = [[int(val >= 0.2) for val in pred] for pred in preds_proba]
# y_true = y_test
# cm = multilabel_confusion_matrix(y_true, y_pred)

In [70]:
# df_onehot_subs = df_onehot

In [71]:
# df_onehot_subs

In [72]:
class ArxivAbstracts(Dataset):
    def __init__(self, df, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.text = df.abstract
        self.labels = df[df.columns.difference(['abstract'])]
        self.max_len = max_len
        
    def __len__(self):
        return len(self.text)
    
    def __getitem__(self, item_idx):
        text = self.text[item_idx]
        encoding = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding="max_length",
            return_token_type_ids=False,
            return_attention_mask= True,
            truncation=True,
            return_tensors='pt'
          )
               
        return {
          'ids': encoding["input_ids"].flatten(),
          'mask': encoding["attention_mask"].flatten(),
          'targets': torch.tensor(self.labels.iloc[item_idx].tolist(), dtype=torch.float)
        }

In [73]:
RANDOM_SEED = 1337
df_train, df_test = train_test_split(df_onehot, test_size=0.1, random_state=RANDOM_SEED, shuffle=True)
df_train, df_validation = train_test_split(df_train, test_size=0.2, random_state=RANDOM_SEED, shuffle=True)
df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)
df_validation = df_validation.reset_index(drop=True)

In [74]:
import pytorch_lightning as pl
class ArxivAbstractsModule(pl.LightningDataModule):
    
    def __init__(self, df_train, df_test, df_validation, tokenizer, batch_size=16, max_token_len=256):
        super().__init__()
        self.df_train = df_train
        self.df_test = df_test
        self.df_validation = df_validation
        self.tokenizer = tokenizer
        self.batch_size = batch_size
        self.max_token_len = max_token_len

    def setup(self, stage=None):
        self.train_dataset = ArxivAbstracts(df=self.df_train, tokenizer=self.tokenizer, max_len= self.max_token_len)
        self.val_dataset = ArxivAbstracts(df=self.df_validation, tokenizer=self.tokenizer, max_len= self.max_token_len)
        self.test_dataset = ArxivAbstracts(df=self.df_test, tokenizer=self.tokenizer, max_len= self.max_token_len)


    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size)

    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size)

    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.batch_size)


In [75]:
tokenizer = AutoTokenizer.from_pretrained('distilroberta-base')
arxiv_data = ArxivAbstractsModule(df_train, df_test, df_validation, tokenizer, max_token_len=384)
arxiv_data.setup()

Downloading:   0%|          | 0.00/480 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

In [76]:
len(arxiv_data.test_dataloader())/16

53.625

In [77]:
class ArxivAbstractsClassifier(pl.LightningModule):
    def __init__(self, config):
        super().__init__()

        self.config = config
        self.embedding = nn.Embedding(100000, 60, padding_idx=1)
        self.gru = nn.GRU(60, 100, batch_first=True, num_layers=3)
        #self.pretrained_model = AutoModel.from_pretrained(config['model_name'], return_dict=True)
        #self.hidden = torch.nn.Linear(self.pretrained_model.config.hidden_size, self.pretrained_model.config.hidden_size)
        self.classifier = torch.nn.Linear(100, 151) 
        #torch.nn.init.xavier_uniform_(self.classifier.weight)
        self.loss_func = torch.nn.BCEWithLogitsLoss(reduction='mean')
        #self.dropout = torch.nn.Dropout()
        self.save_hyperparameters()
    
    def forward(self, ids, mask, targets=None):
        # roberta layer
#         output = self.pretrained_model(input_ids=ids, attention_mask=mask)
#         pooled_output = torch.mean(output.last_hidden_state, 1)
#         # final logits
#         pooled_output = self.dropout(pooled_output)
#         pooled_output = self.hidden(pooled_output)
#         pooled_output = F.relu(pooled_output)
#         pooled_output = self.dropout(pooled_output)
#         logits = self.classifier(pooled_output)
        # calculate loss
        embed = self.embedding(ids)
    
        lengths = torch.sum(mask, axis = 1)
        
        #print(lengths)
            
        packed_embedded = torch.nn.utils.rnn.pack_padded_sequence(embed, lengths.tolist(), batch_first=True, enforce_sorted=False)
    
        output, h_n = self.gru(packed_embedded)
        
        unpacked, _ = torch.nn.utils.rnn.pad_packed_sequence(output, batch_first=True, padding_value=1)
        
#         print(unpacked)
        
        logits = self.classifier(unpacked[range(ids.shape[0]), lengths - 1])
        
        loss = 0
        if targets is not None:
            loss = self.loss_func(logits.view(-1, 151), targets.view(-1, 151))
        return loss, logits
  
    def training_step(self, batch, batch_index):
        loss, outputs = self(**batch)
        self.log("train_loss", loss.item())
        return {"loss":loss, "predictions": outputs, "labels": batch["targets"]}

    def validation_step(self, batch, batch_index):
        loss, outputs = self(**batch)
        self.log("val_loss", loss.item())
        return {"val_loss": loss, "predictions": outputs, "labels": batch["targets"]}

    def predict_step(self, batch, batch_index):
        loss, outputs = self(**batch)
        return torch.sigmoid(outputs)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters())
        return optimizer

In [78]:
checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints",
    monitor='val_loss',
    filename='GRU-{epoch:02d}-{val_loss:.2f}',
    save_top_k=2,
    verbose=True,
    mode='min'
)
logger = TensorBoardLogger("lightning_logs", name="gru2")

config = {
    'model_name': 'distilroberta-base',
    'n_labels': 151,
    'batch_size': 16,
    'lr': 1.5e-6,
    'warmup': 0.2, 
    'train_size': len(arxiv_data.train_dataloader()),
    'weight_decay': 0.001,
    'n_epochs': 30
}


In [79]:
%%time 
model = ArxivAbstractsClassifier(config)
trainer = Trainer(max_epochs = config['n_epochs'], 
                  gpus = 1, 
                  logger=logger, 
                  enable_checkpointing=True, 
                  callbacks=[checkpoint_callback, RichProgressBar(), EarlyStopping(monitor="val_loss", mode="min")])


trainer.fit(model, datamodule=arxiv_data)

Epoch 0    ━━━━╺━━━━━━━━━━━━━━━━━ 1555/7716 0:02:47 • 0:11:04 9.28it/s loss: 0.0765 v_num: 0 

/opt/conda/envs/Python-3.9-CUDA/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer
.py:688: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")

CPU times: user 2min 39s, sys: 19.2 s, total: 2min 58s
Wall time: 2min 49s


In [52]:
model_name = "GRU"
def save_dir_to_cloud(path, model_name):
        for root,dirs,files in os.walk(path):
            for file in files:
                filepath = os.path.join(root,file)
                save_to_cloud(f"outs/{model_name}/{file}", filepath)

In [53]:
trainer.save_checkpoint(f"{model_name}.ckpt")
save_to_cloud(f'{model_name}.ckpt', f"{model_name}.ckpt")

In [54]:
save_dir_to_cloud("lightning_logs", model_name)
save_dir_to_cloud("checkpoints", model_name)

In [55]:
get_file(f'snapshot-{model_name}.ckpt', f"snapshot-{model_name}.ckpt")
# trainer = Trainer()
# trainer.fit(model, datamodule=arxiv_data, ckpt_path=f"snapshot-{model_name}.ckpt")

In [56]:

model = ArxivAbstractsClassifier(config).load_from_checkpoint(f"snapshot-{model_name}.ckpt", config=config)
trainer = Trainer(gpus = 1, 
                  logger=logger,
                  callbacks=[RichProgressBar()])

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [57]:
# trainer.test(model, datamodule=arxiv_data)

In [58]:
preds = trainer.predict(model, dataloaders=arxiv_data.test_dataloader())

Predicting ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 858/858 0:00:51 • 0:00:00 16.56it/s

In [59]:
preds = np.concatenate(preds, axis=0)

In [60]:
# Probably needs a more sophisticated thingy with model.eval + freezing
# Or just incorporating that to the model above so that loading from a checkpoint will work correctly
def predict(tresh = 0.5):
    y_pred = [[int(val >= tresh) for val in pred] for pred in preds]
    y_true = [tensor["targets"] for tensor in arxiv_data.test_dataloader()]
    y_true = np.concatenate([tensor.tolist() for tensor in y_true])
    print(classification_report(y_true, y_pred))
    print("Acc score: ", accuracy_score(y_true, y_pred))

In [ ]:
predict(tresh = 0.35)

In [ ]:
predict(tresh = 0.4)

In [ ]:
predict(tresh = 0.5)

In [ ]:
predict(tresh = 0.2)

In [3]:
%load_ext tensorboard

In [5]:
%tensorboard --event_file=cos://us-geo/arxivabstractstolabelspreds-donotdelete-pr-oehm4wj3avyjr3/snapshot-lightning_logs/arxiv-abstracts-logs/version_0/events.out.tfevents.1654964710.notebook-gpu1rt22199390efe221c4a9c9314e6fc8bfffde0-8645f562s2gc.503.0

ERROR: Failed to launch TensorBoard (exited with 1).
Contents of stderr:
Error: A logdir or db must be specified. For example `tensorboard --logdir mylogdir` or `tensorboard --db sqlite:~/.tensorboard.db`. Run `tensorboard --helpfull` for details and examples.